# Sketch-Map Demo

In [ ]:
from collections import OrderedDict
import numpy as np
import ase.io

from IPython.display import display, clear_output
import ipywidgets as widgets
import traitlets

from subprocess_widget import SubprocessWidget
import bqplot as bq
import nglview

## Step 1: Calculate Sketch-Map

In [ ]:
name = "traj.lys.dipeptide.bare"
w = SubprocessWidget(cmd=["./create_new_sketchmap.sh", name], log_fn="sketchmap.log")
display(w)

## Step 2: Browse Sketch-Map

In [ ]:
def on_load_data(e):
    global all_structs, data
    clear_output()
    print("Loading sketchmap data...")
    data = np.loadtxt(name+"-n10-l8-c3.5-g0.5_fastavg_nnrm.OOS.colvar")
    
    print("Loading structure coordinates...")
    all_structs = ase.io.read(name+".xyz", index=':')
    assert(data.shape[0]==len(all_structs))
    print("Found %d structures"%len(all_structs))
    
    print("Updating scatter plot...")
    with scatter.hold_sync():
        scatter.x = data[:,0]
        scatter.y = data[:,1]
        scatter.color = data[:,2]
        scatter.colvar1 = data[:,2]
        scatter.colvar2 = data[:3]
    print("Done")

btn_load = widgets.Button(description='Load Data')
btn_load.on_click(on_load_data)
display(btn_load)

In [ ]:
# create scatter plot
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()
colvar2_sc = bq.LinearScale()
z_sc = bq.ColorScale()
ax_x = bq.Axis(scale=x_sc)
ax_y = bq.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')
tooltip = bq.Tooltip(fields=['index', 'color'], formats=['','.2e'], labels=["id", "colvar"])
scatter = bq.Scatter(x=[], y=[], color=[],
                     colvar1=[], colvar2=[],
                     scales={'x': x_sc, 'y': y_sc, 'color': z_sc, 'colvar2': colvar2_sc},
                     interactions={'click': 'select', 'hover': 'tooltip'},
                     tooltip=tooltip,
                     selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                     unselected_style={'opacity': 0.5})
fig = bq.Figure(marks=[scatter], axes=[ax_x, ax_y])

# create NGLview
view3D = nglview.NGLWidget()

# selection event handler
def on_select(c):
    # remove old components
    for i in view3D._ngl_component_ids:
        view3D.remove_component(i)
    # add new component if sth is selected
    if scatter.selected != None:
        idx = scatter.selected[-1]
        atoms = all_structs[idx]
        view3D.add_component(nglview.ASEStructure(atoms))
        view3D.add_ball_and_stick()
        view3D.center_view()

# register event handler
scatter.observe(on_select, ['selected'])

# setup PanZoom tool
panzoom = bq.PanZoom(scales={'x': [x_sc], 'y': [y_sc]})
btn_tools = widgets.ToggleButtons(options=OrderedDict([('Select', None), ('Pan + Zoom', panzoom)]))
traitlets.link((btn_tools, 'value'), (fig, 'interaction'))

# display everything
display(widgets.VBox([btn_tools,fig, view3D]))